# Complete Training pipeline 

In this file the training pipeline that we used is displayed. This notebook was not actually used for training, as this was all done on the snellius computer. Python scripts, based on this notebook, were used for that.

## Prepare files and dependencies

Things to change when working in different environment

In [2]:
# dependencies
from datasets import load_dataset, DatasetDict, load_from_disk, Dataset
import os
from transformers import AutoTokenizer, RobertaTokenizerFast
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM
from transformers import pipeline
from transformers import DataCollatorForLanguageModeling
import torch
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path
import json
import re

# general function
def load_json(path):
    with open(path, 'r') as f:
        my_dict = json.load(f)
    return my_dict

# data files
path = '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_short.txt'

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Check that PyTorch sees it
print(f'Cuda available? {torch.cuda.is_available()}')

Cuda available? False


In [3]:
# def create_text_generator(gen):
#     for i in gen:
#         yield i['text']



# oscar_short = load_dataset('text', data_files={"train": path}, split='train')
# oscar_short_it = load_dataset('text', data_files={"train": path}, split='train', streaming=True)

# p = '/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/nl_part_1.txt'

# oscar1_it = load_dataset('text', data_files={"train": p}, split='train', streaming=True)

In [639]:
# # functions for data
# def load_datasets_from_paths(paths):
#     datasets_list = []

#     # Load each dataset from the provided paths
#     for path in paths:
#         dataset = load_dataset('text', data_files={"train": path}, split='train', streaming=True)
#         datasets_list.append(dataset)
    
#     return datasets_list

# def combine_and_batch_datasets(datasets, batch_size=1000):
#     """Yield batches of items from a list of iterable datasets."""
#     for dataset in datasets:
#         batch = []
#         for item in dataset:
#             batch.append(item['text'])
#             if len(batch) == batch_size:
#                 yield batch
#                 batch = []
#         # Yield any remaining items in the current dataset that didn't complete a full batch
#         if batch:
#             yield batch

# #paths = [str(x) for x in Path('/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR').glob("**/*.txt")]


# #combined_generator = combine_and_batch_datasets(load_datasets_from_paths(paths))

In [640]:
# p = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/lower/nl_part_1_lower_short.txt']

# combined_generator1 = combine_and_batch_datasets(load_datasets_from_paths(p))


## Train tokenizer

### BPE

In [75]:
# from pathlib import Path

# from tokenizers import ByteLevelBPETokenizer

# paths = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/lower/nl_part_1_lower_short.txt']

# # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer()

# # Customize training
# tokenizer.train(files=paths, vocab_size=50023, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])

# tokenizer.save_model("/Users/jan/Documents/Master/Thesis/Code/Tokenizers/BPE_NEW")

['/Users/jan/Documents/Master/Thesis/Code/Tokenizers/BPE_NEW/vocab.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/BPE_NEW/merges.txt']

### WordPiece

In [641]:
# # set vocabulary size
# vocab_size = 50023

# # select dataset to train with
# train_set = combined_generator1
# # train_set = create_text_generator(oscar_short_it)

# # load an existing tokenizer
# old_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # retrain the tokenizer
# tokenizerX = old_tokenizer.train_new_from_iterator(train_set, vocab_size)

# tokenizerX.save_pretrained('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX')

('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX/tokenizer_config.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX/special_tokens_map.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX/vocab.txt',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX/added_tokens.json',
 '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX/tokenizer.json')

## Tokenize data

### Load tokenizer

#### Option 1: BPE

In [6]:
from transformers import RobertaTokenizerFast

tokenizerBPE = RobertaTokenizerFast.from_pretrained("/Users/jan/Documents/Master/Thesis/Code/Tokenizers/BPE_NEW", max_len=512)

50005

#### Option 2: WordPiece

In [ ]:
tokenizer_wp = AutoTokenizer.from_pretrained('/Users/jan/Documents/Master/Thesis/Code/Tokenizers/WPX')

#### Option 3: Custom morphological tokenizer

In [55]:

class CustomTokenizer:

    def __init__(self, segmentation_dictionary, wp_tokenizer, max_length=None, pad_to_multiple_of=None, model_max_length=None):



        self.wp_tokenizer = wp_tokenizer
        self.wp_vocab = self.wp_tokenizer.get_vocab()

        self.segmentations = {word: seg for word, seg in segmentation_dictionary.items() if len(seg) > 0}
        self.seg_dict = {}

        
        for word, segs in self.segmentations.items():
            out = []
            for i, seg in enumerate(segs):
                if i == 0:
                    out.append(seg)
                else:
                    out.append('##' + seg)
            self.seg_dict[word] = out
        

        self.segments = {seg for segs in self.seg_dict.values() for seg in segs}

        self.vocab = self.wp_vocab.copy()
        
      

        next_index = len(self.vocab)
        
        for element in self.segments:
            if element not in self.vocab:
                self.vocab[element] = next_index
                next_index += 1


        
        self.vocab_size = len(self.vocab)
        
        self.seg_dict_numbered = {}

        for word, segs in self.seg_dict.items():
            s = []
            for seg in segs:
                s.append(self.vocab[seg])
            self.seg_dict_numbered[word] = s
        
        
        self.inverted_vocab = {value: key for key, value in self.vocab.items()}
        self.max_length = max_length
        self.pad_to_multiple_of = pad_to_multiple_of
        self.padding_side = 'right'
        self.model_max_length = model_max_length

        ### special tokens

        special_tokens = ['[UNK]', '[MASK]', '[CLS]', '[SEP]', '[PAD]']
        special_token_ids = {}
        
        for token in special_tokens:
            if token not in self.vocab:
                self.vocab[token] = len(self.vocab)
            special_token_ids[token] = self.vocab[token]
        
        self.unk_token = '[UNK]'
        self.unk_token_id = special_token_ids['[UNK]']
        
        self.mask_token = '[MASK]'
        self.mask_token_id = special_token_ids['[MASK]']
    
        
        self.pad_token = '[PAD]'
        self.pad_token_id = special_token_ids['[PAD]']
        
        self.bos_token = '[CLS]'
        self.bos_token_id = special_token_ids['[CLS]']
        
        self.eos_token = '[SEP]'
        self.eos_token_id = special_token_ids['[SEP]']
        

        self.special_tokens = [self.vocab['[PAD]'], self.vocab['[UNK]'], self.vocab['[CLS]'], self.vocab['[SEP]'], self.vocab['[MASK]']]

        self.special_tokens_map = wp_tokenizer.special_tokens_map



    def __len__(self):
        return len(self.vocab)


    def check_tokens_in_dict(self, ids, tokens, dic_a):
     
        combined_tokens = []
        current_word = ''
        current_ids = []

        for i, token in enumerate(tokens):
            if token.startswith('##'):
                current_word += token[2:]
                current_ids.append(ids[i])
            else:
                if current_word:
                    combined_tokens.append((current_word, current_ids))
                current_word = token
                current_ids = [ids[i]]

        if current_word:
            combined_tokens.append((current_word, current_ids))
 
        result = []
        for word, ids_list in combined_tokens:
            if word in dic_a:
                result.extend(dic_a[word])  
            else:
                result.extend(ids_list)
        return result
    

    def check_tokens_in_dict_v2(self, ids, tokens, dic_a):

        combined_tokens = []
        current_word = ''
        current_tokens = []

        for i, token in enumerate(tokens):
            if token.startswith('##'):
                current_word += token[2:]
                current_tokens.append(token)
            else:
                if current_word:
                    combined_tokens.append((current_word, current_tokens))
        
                current_word = token
                current_tokens = [token]

        if current_word:
            combined_tokens.append((current_word, current_tokens))
    
        result = []
        for word, tokens_list in combined_tokens:
            if word in dic_a:
                result.extend(dic_a[word])  
            else:
                result.extend(tokens_list)

        return result



    def get_vocab(self):
        return self.vocab




    def _convert_token_to_id(self, token):
        if token in self.vocab:
            return self.vocab[token]
        else:
            return self.unk_token_id

    
    def convert_tokens_to_ids(self, tokens):
        if isinstance(tokens, list):
            return [self._convert_token_to_id(token) for token in tokens]
        return self._convert_token_to_id(tokens)



    def _convert_id_to_token(self, id):
        return self.inverted_vocab[id]


    def convert_ids_to_tokens(self, ids):
        if isinstance(ids, list):
            return [self._convert_id_to_token(id) for id in ids]
        return self._convert_id_to_token(ids)


    def get_special_tokens_mask(self, token_ids, already_has_special_tokens=False):
        return [1 if self._is_special_token(token_id) else 0 for token_id in token_ids]


    def _is_special_token(self, token_id):

        if token_id in self.special_tokens:
            return True
        else:
            return False

    

    def pad(self, batch, return_tensors="pt", pad_to_multiple_of=None, padding=True, max_length=None):
        if pad_to_multiple_of is None:
            pad_to_multiple_of = self.pad_to_multiple_of

        input_ids_list = []
        for dictionary in batch:
            for key, value in dictionary.items():
                if key == "input_ids":
                    
                    if isinstance(value, torch.Tensor):
                        input_ids_list.append(value.tolist())
                    else:
                        input_ids_list.append(value)


        max_length = max(len(x) for x in input_ids_list)
        
        if pad_to_multiple_of is not None:
            max_length = (max_length + pad_to_multiple_of - 1) // pad_to_multiple_of * pad_to_multiple_of
        
        padded_batch = []
        for seq in input_ids_list:
            if len(seq) < max_length:
                seq.extend([self.pad_token_id] * (max_length - len(seq)))
            padded_batch.append(seq)
        
        attention_list = []
        for inner_list in padded_batch:
            p_list = [1 if value != self.pad_token_id else 0 for value in inner_list]
            attention_list.append(p_list)
        
        if return_tensors == "pt":
            return {'input_ids': torch.tensor(padded_batch, dtype=torch.long), 'attention_mask': torch.tensor(attention_list, dtype=torch.long)}
        
        return {'input_ids': padded_batch, 'attention_mask': attention_list}


    def tokenize(self, text):
        if isinstance(text, list):
            tokens_list = []
            for t in text:
                encoded = self.wp_tokenizer(t)
                tokens = encoded.tokens()
                tokens = self.check_tokens_in_dict_v2(encoded['input_ids'], tokens, self.seg_dict)[1:-1]
                tokens_list.append(tokens)
            return tokens_list
        else:
            encoded = self.wp_tokenizer(text)
            tokens = encoded.tokens()
            tokens = self.check_tokens_in_dict_v2(encoded['input_ids'], tokens, self.seg_dict)[1:-1]
            return tokens


    def encode(self, text, text_pair=None, add_special_tokens=True, return_tensors=None, max_length=None, pad_to_max_length=False, truncation=False):
        if text_pair:
            text = f"{text} {self.eos_token} {text_pair}"

        tokens = self.tokenize(text)
        
        if add_special_tokens:
            tokens = [self.bos_token] + tokens + [self.eos_token]
        
        if truncation and max_length and len(tokens) > max_length:
            tokens = tokens[:max_length]
        
        if pad_to_max_length and max_length and len(tokens) < max_length:
            tokens += [self.pad_token] * (max_length - len(tokens))
        
        input_ids = self.convert_tokens_to_ids(tokens)
        attention_mask = [1 if token != self.pad_token else 0 for token in tokens]

        if return_tensors == "pt":
            input_ids = torch.tensor([input_ids], dtype=torch.long)
            attention_mask = torch.tensor([attention_mask], dtype=torch.long)
        
        return {'input_ids': input_ids, 'attention_mask': attention_mask}

    def batch_encode_plus(self, texts, text_pairs=None, add_special_tokens=True, return_tensors=None, max_length=None, pad_to_max_length=False, truncation=False, pad_to_multiple_of=None):
        batch = []

        if text_pairs:
            for text, text_pair in zip(texts, text_pairs):
                batch.append(self.encode(
                    text, 
                    text_pair=text_pair, 
                    add_special_tokens=add_special_tokens, 
                    return_tensors=None, 
                    max_length=max_length, 
                    pad_to_max_length=pad_to_max_length, 
                    truncation=truncation
                ))
        else:
            for text in texts:
                batch.append(self.encode(
                    text, 
                    add_special_tokens=add_special_tokens, 
                    return_tensors=None, 
                    max_length=max_length, 
                    pad_to_max_length=pad_to_max_length, 
                    truncation=truncation
                ))

        padded_batch = self.pad(batch, return_tensors=return_tensors, pad_to_multiple_of=pad_to_multiple_of)

        return padded_batch


    def decode(self, ids, skip_special_tokens=False):
        out = ''
        for id in ids:
            token = self._convert_id_to_token(id)
            if skip_special_tokens and self._is_special_token(id):
                continue
            if token[:2] == '##':
                out += token[2:]
            else:
                out += ' ' + token
        
        
        out = re.sub(r'\s+([?.!,\'"])', r'\1', out)
        return out.strip()


    def __call__(self, text, text_pair=None, add_special_tokens=True, return_tensors=None, max_length=None, pad_to_max_length=False, truncation=False, pad_to_multiple_of=None):
        if isinstance(text, str):
      
            return self.encode(
                text, 
                text_pair=text_pair, 
                add_special_tokens=add_special_tokens, 
                return_tensors=return_tensors, 
                max_length=max_length, 
                pad_to_max_length=pad_to_max_length, 
                truncation=truncation
            )
        elif isinstance(text, list) and all(isinstance(t, str) for t in text):
      
            return self.batch_encode_plus(
                text, 
                text_pairs=text_pair, 
                add_special_tokens=add_special_tokens, 
                return_tensors=return_tensors, 
                max_length=max_length, 
                pad_to_max_length=pad_to_max_length, 
                truncation=truncation, 
                pad_to_multiple_of=pad_to_multiple_of
            )
        else:
            raise ValueError("Input text should be either a single string or a list of strings.")

    
    def __len__(self):
        return self.vocab_size

In [3]:
path_to_dict = '/Users/jan/Documents/Master/Thesis/Code/Tokenizers/segmentation_dictionary_final.json'
path_to_tokenizer = "/Users/jan/Documents/Master/Thesis/Code/Tokenizers/HELP/MORF/help_2815"

segmentation_dictionary = load_json(path_to_dict)
help_tokenizer = AutoTokenizer.from_pretrained(path_to_tokenizer)

tokenizer_morf = CustomTokenizer(segmentation_dictionary, help_tokenizer)

### Create dataset

In our training pipeline, we will not tokenize the data during training. Instead, we first create a Dataset object with the tokenized training data. This ensures that the speed of the tokenizer is nat a bottleneck when training. This could especially be the case for the custom tokenizer, which is a bit slower than the two other ones, although not by that much.

In [ ]:
%%time

import os
import torch
from datasets import Dataset


# pick tokenizer
tokenizer = tokenizer_bpe

# pick text file to transform
text_file_paths = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/Short/OSCAR_shorter2.txt']

#text_file_paths = ['/Users/jan/Documents/Master/Thesis/Code/Datasets/OSCAR/lower/nl_part_1_lower_short.txt']


# def tokenize function
def tokenize_line(line):
    return {'input_ids': tokenizer(line.strip(), add_special_tokens=True, truncation=True, max_length=512)['input_ids']}


# read the text file and tokenize each line

for text_file_path in text_file_paths:
    with open(text_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    tokenized_lines = [tokenize_line(line) for line in lines]

    dataset = Dataset.from_list(tokenized_lines).with_format("torch")
    base_name = text_file_path.strip('.txt')
    base_name = os.path.splitext(os.path.basename(text_file_path))[0]
    dataset_name = f'{base_name}_tokenized'

    p = f'/Users/jan/Documents/Master/Thesis/Code/Datasets/Tokenized/TEST/{dataset_name}'
    print(dataset_name)

    #save
    #dataset.save_to_disk(p)

#### Check dataset

See what a single element in the dataset looks like

In [816]:
dataset[0]

{'input_ids': tensor([    0,    57,  1346,   656,   470,   299,  2452,   557,   576,   314,
           778,    18,   225,    56,   476,   361,   381,   312,  1233,    16,
           518,   361,   557,  3312,   609,   379, 42336,     2])}

Make sure that no element is longer than the max length

In [818]:
def test_length(dataset):
    mx = 0
    for i in range(999):
        if dataset[i]['input_ids'].size()[0] > mx:
            mx = dataset[i]['input_ids'].size()[0]
            n = i
    print('max length:', mx)
    print('at nr', n)

test_length(dataset)


max length: 512
at nr 255


Check whether the special tokens still get added correctly for the truncated elements

In [819]:
dataset[255]

{'input_ids': tensor([    0,    52,   317,  1414,    77, 17104,  2356, 12250,  2110,   423,
           300,  3568,   563,  1049,   396,   842,  3752,    16,   225,    55,
          4489, 20237,    69,  6267,    16,   225,    55,  1532,  5455,   423,
          3733, 41985,    16,   225,    43,   302,  2110,   423,   225,    56,
            58,   443,  2754,   563,   301,   757,    83,   225,    45,    82,
           458,  1059,    69,    16,   225,    42, 10755,  1680,   423,   225,
            38,    45,    51,    55,    17,    69,    16,   225,    47,    83,
         12733,   225,    38,  1630,  1625,  3239,  4437,   459,   408, 23508,
           225,    51,   329,   842,   580,  1054, 23523, 17104,   301,   580,
           267, 38950,    16,   225,    38,   367,  6930,   666,   225,    45,
          2957,    16,   225,    55,    18,    56,    18,    37,    18,    48,
            18,    47,    18,    41,    18,    54,  8021,   225,    55, 12168,
           914,   380,   225,    39, 33

Make sure the dataset works in combination with the data collator we use for training

In [820]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [821]:
def check_data_collator(batch):
    input_ids = batch['input_ids']
    labels = batch['labels']
    pad_token_id = tokenizer.pad_token_id

    # check masked elements
    mask_check = True
    for i in range(input_ids.shape[0]):
        for j in range(input_ids.shape[1]):
            if labels[i][j] != -100:
                if input_ids[i][j] != labels[i][j]:
                    mask_check = False
                    print(f"Mismatch at position [{i}, {j}]: input_id={input_ids[i][j]}, label={labels[i][j]}")

    # check unmasked elements
    unmask_check = True
    for i in range(input_ids.shape[0]):
        for j in range(input_ids.shape[1]):
            if labels[i][j] == -100:
                if input_ids[i][j] == labels[i][j]:
                    unmask_check = False
                    print(f"Unexpected match at position [{i}, {j}]: input_id={input_ids[i][j]}, label={labels[i][j]}")

    # check padding
    padding_check = True
    for i in range(input_ids.shape[0]):
        for j in range(input_ids.shape[1]):
            if input_ids[i][j] == pad_token_id:
                if labels[i][j] != -100:
                    padding_check = False
                    print(f"Padding mismatch at position [{i}, {j}]: input_id={input_ids[i][j]}, label={labels[i][j]}")

    if mask_check and unmask_check and padding_check:
        print("All checks passed.")
    else:
        print("There were some mismatches.")

In [822]:
texts = [dataset[0], dataset[1], dataset[2], dataset[3]]
texts = [dataset[4], dataset[5], dataset[6], dataset[7], dataset[8], dataset[9]]
texts

[{'input_ids': tensor([    0,  1830,   225,    45,    82,  1317,   287,   272,   225,    44,
            334,   225,    46,    73,  4935,   225,    39,  7156,   459,   398,
           3438,   503,  3993,   830, 14758,   310,  4174,   661,   609,   314,
            798,   288,   661,   674,  3979,   314,  6121,    18,     2])},
 {'input_ids': tensor([    0,    59,    73,   360,  1603,   470,   428,   272,  3750,   287,
            272,  2334,   372, 27359,  1262,   293,   557,  1008,  6888,   360,
             16,  1766,   830,   622,   412,  3423, 18796,   396,  1204,   337,
           1397,   524,    16,   288,   674,  3734,  1990,   985,   288,  1539,
           7585,   288, 15432,   361,   677, 47249, 41961,  1535,    18,     2])},
 {'input_ids': tensor([    0,    46,    73,  4935,  5872,  1214,   411,   299,  1008,    18,
            225,    44,   271, 11025,   940,   272, 15666,  1037,  5067,  6116,
             16,   288,   427,  3110,   225,    44,   437,   293,   272,  2847,
  

In [823]:
z = data_collator(texts)

z

{'input_ids': tensor([[    0,  1830,   225,    45,    82,  1317,   287,   272,     4,    44,
           334,   225,     4,    73,  4935,   225,     4,     4,   459,   398,
          3438,     4,  3993,     4, 14758,   310,  4174,   661,   609,   314,
           798,   288,   661,   674,     4,   314,  6121,    18,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,    59,    73,   360,  1603,     4,   428,   272,  3750, 12178,
           272,  2334,   372,     4,  1262,   293,   557,  1008,  6888,   360,
            16,  1766,     4,   622,   412,     4, 18796,   396,  1204,   337,
          1397,     4,    16,     4,   674,  3734,  1990,   985,   288,     4,
          7585,   288, 15432,   361,   677, 47249, 41961,  1535,    18,     2,
             1,     1,     1,     1,     1],
        [    0,    46,    73,  4935,  5872,  1214,   411,   299,  1008,    18,
           225,    44,     

In [813]:
check_data_collator(z)

Mismatch at position [0, 6]: input_id=4, label=2045
Mismatch at position [0, 11]: input_id=4, label=4686
Mismatch at position [0, 16]: input_id=4, label=1457
Mismatch at position [0, 25]: input_id=26359, label=7138
Mismatch at position [1, 2]: input_id=9435, label=1192
Mismatch at position [1, 19]: input_id=4, label=16
Mismatch at position [1, 22]: input_id=4, label=1458
Mismatch at position [1, 25]: input_id=4, label=27854
Mismatch at position [1, 27]: input_id=4, label=1953
Mismatch at position [1, 30]: input_id=4, label=1168
Mismatch at position [1, 45]: input_id=4, label=1267
Mismatch at position [1, 49]: input_id=4, label=18
Mismatch at position [2, 5]: input_id=4, label=1135
Mismatch at position [2, 6]: input_id=4, label=1784
Mismatch at position [2, 9]: input_id=4, label=16508
Mismatch at position [2, 12]: input_id=4, label=16159
Mismatch at position [2, 16]: input_id=4, label=16
Mismatch at position [2, 31]: input_id=4, label=1206
Mismatch at position [3, 7]: input_id=4, label=

These mismatches are all supposed to be there, they are the cosequence of the randomness in the collator

## Initialize model

Select tokenizer

In [35]:
tokenizer = tokenizer_bpe
t_size = len(tokenizer)

Select data collator

In [141]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

Select model configuration

In [797]:

config = RobertaConfig(
    vocab_size=t_size,
    max_position_embeddings=512,
    num_attention_heads=8,
    num_hidden_layers=8,
    type_vocab_size=1, 
    hidden_size=512,
    intermediate_size=1024,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    mask_token_id= tokenizer.mask_token_id
)


model = RobertaForMaskedLM(config=config)

print(f'number of parameters model: {model.num_parameters()}')

number of parameters model: 43011431


In [798]:
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "mask_token_id": 4,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 8,
  "num_hidden_layers": 8,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.33.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50023
}

## Set training arguments

In [460]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/Users/jan/Documents/Master/Thesis/Code/Models/model1", # note: this is not where the model is saved, just info about training
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=1000,
    prediction_loss_only=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

## Train

In [ ]:
%% time

trainer.train()

## Save

In [ ]:
trainer.save_model("/Users/jan/Documents/Master/Thesis/Models/model1")

## Load & Test

In [759]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

In [760]:
fill_mask('ik ga morgen [MASK]')

[{'score': 0.000244285591179505,
  'token': 11434,
  'token_str': '##korting',
  'sequence': 'ik ga morgenkorting'},
 {'score': 0.00023400774807669222,
  'token': 5145,
  'token_str': 'box',
  'sequence': 'ik ga morgen box'},
 {'score': 0.0001880083145806566,
  'token': 1015,
  'token_str': '##ج',
  'sequence': 'ik ga morgenج'},
 {'score': 0.00016760850849095732,
  'token': 16143,
  'token_str': '##hangende',
  'sequence': 'ik ga morgenhangende'},
 {'score': 0.00015900007565505803,
  'token': 24872,
  'token_str': 'avail',
  'sequence': 'ik ga morgen avail'}]

We can see that the untrained model gives random tokens.

## Train again

We want to train our model further. 

In [32]:
# reload model

from transformers import AutoConfig

model_dir = "/Users/jan/Documents/Master/Thesis/model-xx"

config = AutoConfig.from_pretrained(model_dir)

model_trained_reloaded = RobertaForMaskedLM.from_pretrained(
    model_dir,
    config=config,
    use_safetensors=True  # Ensure that the model is loaded from safetensors
)



In [35]:
# test whether the reloaded model returns the same predictions

fill_mask_reloaded = pipeline(
    "fill-mask",
    model=model_trained_reloaded,
    tokenizer=tokenizer
)

fill_mask_untrained2("Ik ga morgen naar <mask>.") == fill_mask("Ik ga morgen naar <mask>.")

True

We have succesfully loaded the trained model. We now want to train it further.

In [ ]:

# reload the tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("/home/scur2141/train_tokenizer1/tokenizer", max_len=512)


training_args = TrainingArguments(
    output_dir="/Users/jan/Documents/Master/Thesis/Code/Models/model2", 
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model_trained_reloaded, 
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)
